In [1]:
%%capture
!pip install gradio
!pip install accelerate -U
!pip install transformers huggingface_hub
!pip install gliner[gpu]

In [11]:
from gliner import GLiNER
import torch

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(device)

model = GLiNER.from_pretrained("numind/NuNER_Zero").to(device)

cuda


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.80G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.80G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

...

NuZero_token_token_metrics.txt:   0%|          | 0.00/961 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/venv/main/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
label_mapping = {
    "ORG": "organization",
    "PERS": "person",
    "LOC": "location",
    "MON": "monetary value",
    "PCT": "percentage",
    "DATE": "date",
    "TIME": "timestamp",
    "PERIOD": "time period",
    "JOB": "job title",
    "DOC": "document name",
    "QUANT": "quantity",
    "ART": "artifact",
    "MISC": "miscellaneous"
}

labels = list(label_mapping.values())
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

In [12]:
def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['label'] == current['label'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['text'] = text[current['start']: next_entity['end']].strip()
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    # Append the last entity
    merged.append(current)
    return merged

In [13]:
from tqdm import tqdm
import pandas as pd
import json

df = pd.read_csv("test.csv")

results = []
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    _, text, _ = row
    entities = model.predict_entities(text, labels)

    entities = merge_entities(entities)
    
    formatted_entities = [{
        "label": reverse_label_mapping[ent["label"]], 
        "text": ent["text"],
        "start": ent["start"],
        "end": ent["end"]
    } for ent in entities]
    
    results.append(json.dumps(formatted_entities, ensure_ascii=False))
    
df["pred"] = results

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|██████████| 5087/5087 [01:16<00:00, 66.86it/s]


In [14]:
df.to_csv("NuNER_Zero.csv", index=False)